<a href="https://colab.research.google.com/github/awildt01/Airbnb_Berlin-/blob/main/notebooks/3__select_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zumanenfassung

## **Verwendete Methode zur Feature-Auswahl**

In diesem Schritt wird eine Kombination aus **manueller Feature-Selektion** und der **Vermeidung der Dummy-Variablen-Falle** angewendet.



### **Manuelle Feature-Selektion**

Die Auswahl der über 100 verwendeten Variablen wurde bereits in einem vorherigen Notebook durchgeführt.
Dabei kam kein automatisierter Algorithmus zum Einsatz, sondern eine gezielte Analyse durch den Analysten.
Zu den eingesetzten Methoden zählen unter anderem:

- **explorative Datenanalyse (EDA)**

- **Berechnung des Information Value (IV)**

- **Analyse des Weight of Evidence (WoE)**

Auf dieser Grundlage wurde entschieden, welche Variablen für das Modell am relevantesten sind.



### **Vermeidung der Dummy-Variablen-Falle**

Bei kategorialen Variablen mit *k* Kategorien entstehen nach dem One-Hot-Encoding *k* Dummy-Variablen.
Wenn **alle** Dummy-Variablen verwendet werden, führt das zu **perfekter Multikollinearität**.
Dies ist insbesondere bei **linearer** und **logistischer Regression** problematisch.

**Lösung:**
Aus jeder Gruppe von Dummy-Variablen wird **eine Kategorie entfernt**.
Diese entfernte Variable wird zur **Referenzkategorie**, die als Vergleichsbasis dient.

**Beispiel:**
Im Code wird die Variable **term:60** (Laufzeit: 60 Monate) entfernt.
Der Koeffizient von **term:36** misst dann den Unterschied im Kreditrisiko im Vergleich zur Referenz **term:60**.



**Fazit**

Die eigentliche Feature-Auswahl wurde bereits in einem vorherigen Schritt getroffen.
Der hier gezeigte Code sorgt dafür, dass die gewählten Variablen korrekt für die Modellierung vorbereitet werden, indem er **Multikollinearität vermeidet** und gleichzeitig eine **Referenzkategorie für die Interpretation** festlegt.







## **1. Vorbereitung der Eingabedaten**

**Feature-Auswahl und Referenzkategorien für die logistische Regression**

**1- Auswahl der Features (Variablenauswahl):** Zuerst wird ein neuer DataFrame **inputs_train_with_ref_cat** erstellt. Dieser enthält nur eine spezifische Auswahl von Spalten (Features) aus dem ursprünglichen Trainingsdatensatz **loan_data_inputs_train**. Die ausgewählten Spalten sind bereits in **"Dummy-Variablen"** umgewandelt worden. Das bedeutet, dass kategoriale Variablen wie grade (Kreditwürdigkeitsstufe) oder home_ownership (Wohneigentum-Status) in mehrere binäre (0/1) Spalten aufgeteilt wurden.

**2- Definition der Referenzkategorien:** Eine Liste namens **ref_categories** wird erstellt. Diese Liste enthält die Namen von Dummy-Variablen, die als "Referenzkategorien" dienen sollen. Für jede ursprüngliche kategoriale Variable (z.B. grade) wird eine ihrer Dummy-Variablen (hier grade:G) in diese Liste aufgenommen.

**3- Entfernen der Referenzkategorien:** Die in der **ref_categories-Liste** definierten Spalten werden aus dem DataFrame inputs_train_with_ref_cat entfernt. Das Ergebnis ist ein neuer, finaler DataFrame inputs_train, der für das Modelltraining verwendet wird.
Zusammenfassend lässt sich sagen: Der Code selektiert eine Reihe von vorverarbeiteten Features und entfernt dann aus jeder kategorialen Gruppe eine Referenzkategorie, um Multikollinearität zu vermeiden.

Zusammenfassend ist die Definition einer Referenzkategorie eine technische Notwendigkeit, um **Multikollinearität zu vermeiden**, und gleichzeitig ein mächtiges Werkzeug für die Interpretation, da sie den Bezugsrahmen für die Bewertung aller anderen Kategorien festlegt.

### **1.1 Was ist eine Referenzkategorie?**

Eine Referenzkategorie ist die Basiskategorie, mit der alle anderen Kategorien einer kategorialen Variable verglichen werden, wenn sie in einem statistischen Modell (wie der logistischen oder linearen Regression) verwendet wird.
Stellen Sie sich vor, Sie möchten den Einfluss der home_ownership (Wohneigentum-Status) auf das Kreditrisiko untersuchen. Diese Variable hat drei mögliche Ausprägungen (Kategorien): MORTGAGE (Hypothek), OWN (Eigentum) und RENT (Miete).
Ein Computer kann mit den Textwerten "MORTGAGE" oder "RENT" nicht direkt rechnen. Deshalb wandeln wir sie in Dummy-Variablen um. Das sieht so aus:

| home_ownership | home_ownership:MORTGAGE | home_ownership:OWN | home_ownership:RENT |
|----------------|-------------------------|--------------------|---------------------|
| MORTGAGE       | 1                       | 0                  | 0                   |
| OWN            | 0                       | 1                  | 0                   |
| RENT           | 0                       | 0                  | 1                   |



**Das Problem: Die "Dummy-Variablen-Falle" (Multikollinearität)**

Wenn wir alle drei Dummy-Variablen in ein Regressionsmodell aufnehmen, stoßen wir auf ein Problem namens perfekte Multikollinearität. Das bedeutet, dass eine Variable perfekt aus den anderen vorhergesagt werden kann.
  - Beispiel: Wenn home_ownership:MORTGAGE = 0 und home_ownership:OWN = 0 ist, dann muss home_ownership:RENT = 1 sein. Es gibt keine andere Möglichkeit.

Diese redundante Information verwirrt das Modell. Es kann keine eindeutigen Koeffizienten für jede Kategorie berechnen, weil ihre Effekte nicht voneinander zu trennen sind. Mathematisch führt dies zu einem Fehler.



**Die Lösung:**

Eine Kategorie als Referenzpunkt festlegen
Um dieses Problem zu lösen, lassen wir einfach eine der Dummy-Variablen weg. Die weggelassene Kategorie wird zur Referenzkategorie.
In Ihrem Code wurde für home_ownership die Kategorie RENT_OTHER_NONE_ANY als Referenz gewählt. Nehmen wir an, wir vereinfachen das auf RENT. Dann sieht der Datensatz, den das Modell tatsächlich verwendet, so aus:

| home_ownership | home_ownership:MORTGAGE | home_ownership:OWN |
|----------------|-------------------------|--------------------|
| MORTGAGE       | 1                       | 0                  |
| OWN            | 0                       | 1                  |
| RENT           | 0                       | 0                  |

**Wie wird das jetzt interpretiert?**

- Ein Kreditnehmer mit MORTGAGE wird durch home_ownership:MORTGAGE = 1 identifiziert.
- Ein Kreditnehmer mit OWN wird durch home_ownership:OWN = 1 identifiziert.
- Ein Kreditnehmer mit RENT (die Referenzkategorie) wird dadurch identifiziert, dass beide anderen Spalten 0 sind. Er ist der "Standardfall".

**Die Bedeutung im Modell:** Alles ist relativ

Wenn das Modell trainiert wird, berechnet es Koeffizienten (Gewichte) für home_ownership:MORTGAGE und home_ownership:OWN. Diese Koeffizienten beschreiben den Unterschied im Effekt im Vergleich zur Referenzkategorie (RENT).
Angenommen, das Modell liefert folgende (fiktive) Koeffizienten für die Wahrscheinlichkeit eines Kreditausfalls:
- home_ownership:MORTGAGE: -0.25
- home_ownership:OWN: -0.10

**Interpretation:**

- Der negative Koeffizient von -0.25 für MORTGAGE bedeutet: Kreditnehmer mit einer Hypothek haben ein geringeres Kreditausfallrisiko als Kreditnehmer, die zur Miete wohnen.
- Der Koeffizient von -0.10 für OWN bedeutet: Kreditnehmer mit Eigenheim haben ebenfalls ein geringeres Risiko als Mieter, aber der risikomindernde Effekt ist nicht so stark wie bei Hypothekennehmern.

Die Referenzkategorie (RENT) hat keinen eigenen Koeffizienten. Ihr Effekt ist sozusagen im "Grundrauschen" des Modells (dem Intercept oder Achsenabschnitt) enthalten. Sie ist der Nullpunkt, von dem aus die anderen Effekte gemessen werden.

### **1.2 Wie wählt man die Referenzkategorie aus?**

Die Wahl der Referenzkategorie ist eine strategische Entscheidung. Übliche Vorgehensweisen sind:
1. **Die häufigste Kategorie:** Das macht die Interpretation der anderen Koeffizienten oft intuitiv, da sie den Unterschied zur "normalsten" Gruppe zeigen.
2. **Die Kategorie mit dem geringsten Risiko (oder Erfolg):** Wenn man z.B. grade:G (die schlechteste Bonitätsnote) als Referenz wählt, werden alle anderen Koeffizienten (grade:A, grade:B etc.) positiv sein und zeigen, um wie viel besser diese Noten im Vergleich zur schlechtesten sind. Dies ist in Ihrem Code der Fall und eine sehr gängige Praxis in der Kreditrisikomodellierung.
3. **Eine logische "Basis"-Kategorie:** Zum Beispiel könnte man bei einer medizinischen Studie die "Placebo"-Gruppe als Referenz wählen.

**Zusammenfassung des Workflows: Von Koeffizienten zur Variablenselektion**

**Ziel:**  
Die Identifikation und Beibehaltung **nur der statistisch signifikanten Variablen** im finalen PD-Modell.

---

**Ablauf:**

1.  **Berechnung der Koeffizienten:**
    *   Der erste Schritt ist die Schätzung des Modells, um die **Koeffizienten** (Gewichte) für jede Dummy-Variable zu erhalten.
    *   Diese Koeffizienten zeigen Richtung und Stärke des Einflusses einer Variable auf die Ausfallwahrscheinlichkeit.

2.  **Berechnung der p-Werte:**
    *   Im nächsten Schritt werden die **multivariaten p-Werte** für jeden dieser Koeffizienten berechnet.
    *   Die p-Werte dienen als "Realitäts-Check". Sie quantifizieren, ob ein gefundener Effekt statistisch signifikant ist oder nur zufällig auftreten könnte.

3.  **Selektion der Variablengruppen (der entscheidende Schritt):**
    *   Anhand der p-Werte werden nicht einzelne Dummy-Variablen, sondern **ganze Gruppen** von Dummy-Variablen bewertet, die zusammen eine ursprüngliche Variable (z.B. `grade`) repräsentieren.
    *   **Entscheidungsregel:**
        *   **Behalten, wenn:** Mindestens eine Dummy-Variable in der Gruppe einen signifikanten p-Wert (p < 0.05) aufweist.
        *   **Entfernen, wenn:** **Keine einzige** Dummy-Variable in der Gruppe einen signifikanten Effekt hat.

---

**Warum dieser Ansatz?**  
Diese Methodik stellt sicher, dass das finale Modell nicht nur statistisch robust ist, sondern auch inhaltlich-logisch interpretierbar bleibt. Eine ursprüngliche Variable (wie das Einkommen) wird entweder vollständig im Modell behalten oder vollständig entfernt. Dies verhindert ein unvollständiges oder verzerrtes Bild der Realität.

**Ergebnis:**  
Ein schlankes, stabiles und aussagekräftiges Modell, das nur die stärksten und zuverlässigsten Prädiktoren für die Kreditausfallwahrscheinlichkeit enthält.

**Kurz gesagt:**  
Die Schätzung der Koeffizienten ist die Grundlage, um die p-Werte zu berechnen, welche uns wiederum die Selektionsentscheidung für die Variablengruppen erlauben.

 ### **1.3 Loading the data and selecting the features**

#### **1.3.1 Import Libraries**

In [1]:
import numpy as np
import pandas as pd

from google.colab import drive
import os

In [2]:
# Google Drive mounten
drive.mount('/content/drive')

# Zielordner definieren
folder_path = '/content/drive/MyDrive/Lending-Club-Credit-Scoring/notebooks'
os.makedirs(folder_path, exist_ok=True)  # Ordner erstellen, falls nicht vorhanden

Mounted at /content/drive


In [3]:
loan_data_inputs_train = pd.read_csv('/content/drive/MyDrive/Lending-Club-Credit-Scoring/notebooks/loan_data_inputs_train.csv', index_col = 0,)

In [4]:
loan_data_targets_train = pd.read_csv('/content/drive/MyDrive/Lending-Club-Credit-Scoring/notebooks/loan_data_targets_train.csv', index_col = 0,header=0)

In [5]:
loan_data_inputs_test = pd.read_csv('/content/drive/MyDrive/Lending-Club-Credit-Scoring/notebooks/loan_data_inputs_test.csv',index_col = 0,)

In [6]:
loan_data_targets_test = pd.read_csv('/content/drive/MyDrive/Lending-Club-Credit-Scoring/notebooks/loan_data_targets_test.csv',index_col = 0, header=0)

#### **1.3.2 Explore Data**

In [7]:
pd.set_option('display.max_info_rows', 300)

loan_data_inputs_train.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
427211,292984,29074404,31607606,11250,11250,11250.0,60 months,16.99,279.54,D,...,0,0,0,1,0,0,0,0,0,0
206088,57793,9026077,10838105,8000,8000,8000.0,36 months,18.55,291.44,D,...,0,1,0,1,0,0,0,0,0,0
136020,140700,4865088,6157358,6000,6000,6000.0,36 months,10.16,194.06,B,...,0,0,0,1,0,0,0,0,0,0
412305,297571,28764454,31297687,8000,8000,8000.0,36 months,9.17,255.04,B,...,0,0,0,1,0,0,0,0,0,0
36159,5264,985083,1208540,5000,5000,5000.0,36 months,11.71,165.38,B,...,0,0,0,1,0,0,0,0,0,0


In [8]:
loan_data_targets_train.head()

,good_bad
427211,1
206088,1
136020,1
412305,1
36159,0


In [9]:
loan_data_inputs_train.shape

(373028, 330)

In [10]:
loan_data_targets_train.shape

(373028, 1)

In [11]:
for i, col in enumerate(loan_data_inputs_train.columns, 1):
    print(f"{i}. {col}")

1. Unnamed: 0
2. id
3. member_id
4. loan_amnt
5. funded_amnt
6. funded_amnt_inv
7. term
8. int_rate
9. installment
10. grade
11. sub_grade
12. emp_title
13. emp_length
14. home_ownership
15. annual_inc
16. verification_status
17. issue_d
18. loan_status
19. pymnt_plan
20. url
21. desc
22. purpose
23. title
24. zip_code
25. addr_state
26. dti
27. delinq_2yrs
28. earliest_cr_line
29. inq_last_6mths
30. mths_since_last_delinq
31. mths_since_last_record
32. open_acc
33. pub_rec
34. revol_bal
35. revol_util
36. total_acc
37. initial_list_status
38. out_prncp
39. out_prncp_inv
40. total_pymnt
41. total_pymnt_inv
42. total_rec_prncp
43. total_rec_int
44. total_rec_late_fee
45. recoveries
46. collection_recovery_fee
47. last_pymnt_d
48. last_pymnt_amnt
49. next_pymnt_d
50. last_credit_pull_d
51. collections_12_mths_ex_med
52. mths_since_last_major_derog
53. policy_code
54. application_type
55. annual_inc_joint
56. dti_joint
57. verification_status_joint
58. acc_now_delinq
59. tot_coll_amt
60. 

#### 1.3.3 Selecting the Features

In [12]:
inputs_train_with_ref_cat = loan_data_inputs_train.loc[: , ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',

'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',

 #'addr_state:ND',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:OK_LA_NC_NM_MO_VA_NJ',
'addr_state:MD_TN_AZ_PA_MI',
'addr_state:DE_AR',
'addr_state:UT_MN_OH_IN_GA_ RI_WA',
'addr_state:OR_KY_MA',
'addr_state:MT_MS_SD',
'addr_state:WI_IL_CT_AK',
'addr_state:CO_SC',
'addr_state:KS_NH_WV_VT_ID',
'addr_state:WY_DC_ME',

'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',

'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',

'initial_list_status:f',
'initial_list_status:w',

'term:36',
'term:60',

'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',

'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',

'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',

'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',


'delinq_2yrs:0',
'delinq_2yrs:1-3',
'delinq_2yrs:4-16',
'delinq_2yrs:>=17',


'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',


'open_acc:0',
'open_acc:1-3',
'open_acc:4-12',
'open_acc:13-17',
'open_acc:18-22',
'open_acc:23-25',
'open_acc:26-30',
'open_acc:>=31',

'pub_rec:0-2',
'pub_rec:3-4',
'pub_rec:>=5',

'total_acc:0-15',
'total_acc:16-70',
'total_acc:71-90',
'total_acc:>90',

'acc_now_delinq:0',
'acc_now_delinq:>=1',

'total_rev_hi_lim:<=5K',
'total_rev_hi_lim:5K-10K',
'total_rev_hi_lim:10K-20K',
'total_rev_hi_lim:20K-30K',
'total_rev_hi_lim:30K-40K',
'total_rev_hi_lim:40K-55K',
'total_rev_hi_lim:55K-95K',



'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',

'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',


'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',

'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86',

]]

#### 1.3.4 Referenzkategorie-Dummyvariablen:

- Hier speichern wir die Namen der Referenzkategorie-Dummyvariablen in einer Liste.

In [13]:
ref_categories = ['grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
#'addr_state:ND',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'delinq_2yrs:0',
'inq_last_6mths:>6',
'open_acc:0',
'pub_rec:0-2',
'total_acc:0-15',
'acc_now_delinq:0',
'total_rev_hi_lim:<=5K',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

- Bei kategorialen Variablen mit k Kategorien entstehen nach dem One-Hot-Encoding k Dummy-Variablen.
- Wenn alle Dummy-Variablen beibehalten werden, führt das zu perfekter Multikollinearität, was insbesondere bei linearer und logistischer Regression problematisch ist.

- Aus jeder Gruppe von Dummy-Variablen wird eine Kategorie entfernt.
Diese entfernte Variable wird zur Referenzkategorie, die als Vergleichsbasis dient.

In [14]:
# Referenzkategorie-Dummyvariablen werden raugenommen aus.
pd.set_option('display.max_info_rows', 100)
inputs_train = inputs_train_with_ref_cat.drop(ref_categories, axis = 1)
inputs_train.sample(100)

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:OK_LA_NC_NM_MO_VA_NJ,addr_state:MD_TN_AZ_PA_MI,...,dti:22.4-35,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
342687,True,False,False,False,False,False,False,True,0,1,...,0,1,0,0,0,0,0,0,0,0
399230,False,False,True,False,False,False,False,True,0,0,...,0,0,1,0,0,0,0,1,0,0
202498,False,True,False,False,False,False,False,True,0,0,...,0,1,0,0,0,0,0,0,0,0
167153,True,False,False,False,False,False,False,True,1,0,...,0,1,0,0,0,0,0,0,0,0
244995,False,False,True,False,False,False,False,True,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386492,False,False,True,False,False,False,False,False,0,1,...,0,1,0,0,0,0,0,0,0,0
314043,False,True,False,False,False,False,False,True,0,0,...,0,0,0,0,1,0,0,0,0,0
423193,True,False,False,False,False,False,False,False,0,0,...,0,1,0,0,0,0,0,0,0,0
154181,False,False,False,False,True,False,False,True,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
# Spalten Zeigen
for i, col in enumerate(inputs_train.columns, 1):
    print(f"{i}. {col}")

1. grade:A
2. grade:B
3. grade:C
4. grade:D
5. grade:E
6. grade:F
7. home_ownership:OWN
8. home_ownership:MORTGAGE
9. addr_state:OK_LA_NC_NM_MO_VA_NJ
10. addr_state:MD_TN_AZ_PA_MI
11. addr_state:DE_AR
12. addr_state:UT_MN_OH_IN_GA_ RI_WA
13. addr_state:OR_KY_MA
14. addr_state:MT_MS_SD
15. addr_state:WI_IL_CT_AK
16. addr_state:CO_SC
17. addr_state:KS_NH_WV_VT_ID
18. addr_state:WY_DC_ME
19. verification_status:Not Verified
20. verification_status:Source Verified
21. purpose:credit_card
22. purpose:debt_consolidation
23. purpose:oth__med__vacation
24. purpose:major_purch__car__home_impr
25. initial_list_status:w
26. term:36
27. emp_length:1
28. emp_length:2-4
29. emp_length:5-6
30. emp_length:7-9
31. emp_length:10
32. mths_since_issue_d:<38
33. mths_since_issue_d:38-39
34. mths_since_issue_d:40-41
35. mths_since_issue_d:42-48
36. mths_since_issue_d:49-52
37. mths_since_issue_d:53-64
38. mths_since_issue_d:65-84
39. int_rate:<9.548
40. int_rate:9.548-12.025
41. int_rate:12.025-15.74
42. in

## **2. PD Model Estimation**

In diesem Schritt wird ein logistisches Regressionsmodell geschätzt, um die Wahrscheinlichkeit eines Kreditausfalls (Probability of Default, PD) zu modellieren.
Das Ziel ist nicht, bereits das finale Modell zu bauen, sondern die Koeffizienten (Gewichte) der ausgewählten Variablen zu berechnen und den Zusammenhang zwischen den Merkmalen und dem Ausfallrisiko sichtbar zu machen.
Die geschätzten Koeffizienten dienen als Grundlage für die spätere Modellvalidierung und für den Aufbau einer Scorecard.

**Interpretation der Koeffizienten:**

- Ein positiver Koeffizient bedeutet, dass mit zunehmendem Wert dieser Variable das Risiko eines Ausfalls steigt.
Beispiel: Wenn die Variable „delinq_2yrs“ (Anzahl früherer Zahlungsausfälle) einen positiven Koeffizienten hat, dann erhöht jeder zusätzliche Zahlungsausfall die Ausfallwahrscheinlichkeit.

- Ein negativer Koeffizient bedeutet, dass mit zunehmendem Wert dieser Variable das Risiko eines Ausfalls sinkt.
Beispiel: Hat die Variable „annual_inc“ (Jahreseinkommen) einen negativen Koeffizienten, so reduziert ein höheres Einkommen die Ausfallwahrscheinlichkeit.

Die Schätzung der Koeffizienten im PD-Modell ist ein wichtiger Schritt, da sie es uns ermöglicht, die Beziehung zwischen den unabhängigen Variablen (z. B. demographische Daten, Kreditmerkmale) und der Wahrscheinlichkeit eines Kreditausfalls zu quantifizieren.

Hier sind die Schritte, wie die Koeffizienten ermittelt werden:

1- **Logistische Regression:** Das PD-Modell wird in der Regel mit einer logistischen Regression geschätzt, da diese Methode gut geeignet ist, um Wahrscheinlichkeiten für binäre Ergebnisse (z. B. Ausfall oder kein Ausfall) vorherzusagen.

2- **Odss-Verhältnis:** Bei der logistischen Regression wird das Verhältnis der Wahrscheinlichkeiten (Odds) für die zwei möglichen Ergebnisse (z. B. Ausfall vs. Nicht-Ausfall) analysiert. Die logarithmische Umwandlung dieser Wahrscheinlichkeiten ermöglicht die Linearität, die für die Regression erforderlich ist.

3- **Koeffizienten:** Die Koeffizienten, die durch das Schätzen des Modells erhalten werden, repräsentieren die Veränderung des logarithmischen Odds der abhängigen Variable (z. B. Ausfall) entsprechend einer Einheit Veränderung der jeweiligen unabhängigen Variablen. Ein positiver Koeffizient deutet darauf hin, dass mit steigenden Werten der unabhängigen Variablen die Wahrscheinlichkeit eines Ausfalls steigt, während ein negativer Koeffizient das Gegenteil anzeigt.

**Interpretation der Koeffizienten:**

- Ein positiver Koeffizient bedeutet, dass mit zunehmendem Wert dieser Variable das Risiko eines Ausfalls steigt.
Beispiel: Wenn die Variable „delinq_2yrs“ (Anzahl früherer Zahlungsausfälle) einen positiven Koeffizienten hat, dann erhöht jeder zusätzliche Zahlungsausfall die Ausfallwahrscheinlichkeit.

- Ein negativer Koeffizient bedeutet, dass mit zunehmendem Wert dieser Variable das Risiko eines Ausfalls sinkt.
Beispiel: Hat die Variable „annual_inc“ (Jahreseinkommen) einen negativen Koeffizienten, so reduziert ein höheres Einkommen die Ausfallwahrscheinlichkeit.

Zusammengefasst dient die PD-Modellschätzung nicht nur der Vorhersage, sondern auch der Risikoanalyse und Entscheidungsfindung im Kreditgeschäft.

Vorgehensweise:

#### **2.1 Modellinitialisierung:**

- Import der Klasse **LogisticRegression** aus sklearn.linear_model.

- Erstellen einer Instanz dieser Klasse: **reg = LogisticRegression()**.


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [20]:
# We create an instance of an object from the 'LogisticRegression' class.
reg = LogisticRegression()


#### **2.2 Konfiguration der Anzeige:**

- Um die vollständige Ausgabe aller Ergebnisse sicherzustellen, wird die Anzeigeoption von pandas so eingestellt, dass alle Zeilen angezeigt werden (pd.set_option('display.max_rows', None)).

In [21]:
pd.options.display.max_rows = None
# Sets the pandas dataframe options to display all columns/ rows.

#### **2.3 Modellschätzung:**

- Das Modell wird mit den Trainingsdaten angepasst: reg.fit(inputs_train, loan_data_targets_train).

- **inputs_train** ist der DataFrame, der alle unabhängigen Dummy-Variablen ohne die Referenzkategorien enthält.

- loan_data_targets_train ist der DataFrame, der die abhängige Variable (Zielvariable) „good/bad“ enthält.

In [22]:
reg.fit(inputs_train, loan_data_targets_train)
# Estimates the coefficients of the object from the 'LogisticRegression' class
# with inputs (independent variables) contained in the first dataframe
# and targets (dependent variables) contained in the second dataframe.

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

#### **2.4 Extraktion der Koeffizienten:**

- Der Achsenabschnitt (Intercept) wird mit reg.intercept_ abgerufen.

- Die Koeffizienten der unabhängigen Variablen werden mit reg.coef_ abgerufen. Diese sind in der gleichen Reihenfolge wie die Spalten in inputs_train angeordnet.

In [23]:
# Displays the intercept contain in the estimated ("fitted") object from the 'LogisticRegression' class.
reg.intercept_

array([-0.22888533])

In [24]:
reg.coef_
# Displays the coefficients contained in the estimated ("fitted") object from the 'LogisticRegression' class.

array([[ 0.91379219,  0.6810852 ,  0.48525663,  0.31865566,  0.18176718,
         0.0539709 ,  0.08339154,  0.11849199, -0.01883835,  0.00495697,
         0.0277987 ,  0.07478424,  0.07778338,  0.14437675,  0.2077891 ,
         0.22516539,  0.28307281,  0.62601643,  0.08871873, -0.01444945,
         0.28992394,  0.17956721,  0.20506786,  0.24224472,  0.04019079,
         0.0611153 ,  0.1201992 ,  0.10780958,  0.06599999,  0.05984226,
         0.11821753,  0.91784983,  0.79520111,  0.71147941,  0.53797731,
         0.42347469,  0.20558765, -0.02189845,  0.99598279,  0.64695559,
         0.40138435,  0.17241209,  0.07867354,  0.06415454,  0.10513601,
         0.14344511,  0.15745962, -0.06782673, -0.12201778, -0.02130859,
         0.30871449,  0.17103258, -0.01903637,  0.03766524, -0.01562803,
        -0.02536233, -0.05196009, -0.0562118 , -0.03236376, -0.03047309,
        -0.00664533,  0.07801373, -0.04977451, -0.09052146, -0.00662704,
         0.18466315, -0.01664752, -0.02374649, -0.0

#### **2.5 Erstellung einer Ergebnistabelle:**

- Die Namen der unabhängigen Variablen werden aus den Spalten von inputs_train in einer Variable feature_name gespeichert.

- Ein DataFrame summary_table wird erstellt, mit einer Spalte Feature name (aus feature_name) und einer Spalte Coefficients (die transponierten Koeffizienten aus reg.coef_ mittels numpy).

- **Hinzufügen des Intercepts:**

- Der Intercept wird als erste Zeile in die summary_table eingefügt. Dazu werden zunächst die Indizes aller vorhandenen Zeilen um 1 erhöht, um Platz für den Intercept-Eintrag mit dem Index 0 zu schaffen.

- Die erste Zeile (Index 0) wird mit den Werten Intercept und dem tatsächlichen Intercept-Wert befüllt.

- Abschließend wird die Tabelle nach dem Index sortiert, um den Intercept an erster Stelle zu halten.

In [25]:
feature_name = inputs_train.columns.values
# Stores the names of the columns of a dataframe in a variable.

In [26]:
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
# Creates a dataframe with a column titled 'Feature name' and row values contained in the 'feature_name' variable.
summary_table['Coefficients'] = np.transpose(reg.coef_)
# Creates a new column in the dataframe, called 'Coefficients',
# with row values the transposed coefficients from the 'LogisticRegression' object.
summary_table.index = summary_table.index + 1
# Increases the index of every row of the dataframe with 1.
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
# Assigns values of the row with index 0 of the dataframe.
summary_table = summary_table.sort_index()
# Sorts the dataframe by index.
summary_table

,Feature name,Coefficients
0,Intercept,-0.228885
1,grade:A,0.913792
2,grade:B,0.681085
3,grade:C,0.485257
4,grade:D,0.318656
5,grade:E,0.181767
6,grade:F,0.053971
7,home_ownership:OWN,0.083392
8,home_ownership:MORTGAGE,0.118492
9,addr_state:OK_LA_NC_NM_MO_VA_NJ,-0.018838


In [ ]:
bool_cols = inputs_train.select_dtypes(include=['bool']).columns
inputs_train[bool_cols] = inputs_train[bool_cols].astype(int)

#### **2.6. Ergebnis:**

- Die summary_table bietet eine übersichtliche Darstellung aller Modellkoeffizienten (einschließlich Intercept) zusammen mit den zugehörigen Variablennamen und dient als Grundlage für die weitere Analyse und Interpretation des Modells.

### Korr_matriz

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

matrix = inputs_train.corr()

matrix

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:OK_LA_NC_NM_MO_VA_NJ,addr_state:MD_TN_AZ_PA_MI,...,dti:22.4-35,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
grade:A,1.000000,-0.281766,-0.264622,-0.193788,-0.126141,-0.074456,-0.005355,0.065822,-0.001681,-0.007343,...,-0.086285,0.128550,-0.089807,-0.043991,-0.030146,-0.014320,-0.020827,-0.072079,-0.023405,-0.053254
grade:B,-0.281766,1.000000,-0.391304,-0.286559,-0.186528,-0.110100,-0.001790,0.005986,-0.002915,-0.003230,...,-0.036319,0.009747,-0.009788,0.001302,0.001143,-0.009835,-0.004384,-0.015251,-0.003582,0.018410
grade:C,-0.264622,-0.391304,1.000000,-0.269123,-0.175179,-0.103401,0.000212,-0.016001,0.001653,0.002688,...,0.032247,-0.038853,0.029646,0.010905,0.010037,0.009784,0.009233,0.040123,0.009529,0.017134
grade:D,-0.193788,-0.286559,-0.269123,1.000000,-0.128287,-0.075723,0.003670,-0.035240,0.001370,0.002945,...,0.048506,-0.049909,0.032027,0.019243,0.010826,0.009879,0.010159,0.030623,0.011940,0.006545
grade:E,-0.126141,-0.186528,-0.175179,-0.128287,1.000000,-0.049290,0.003561,-0.015168,0.001286,0.006287,...,0.041550,-0.038756,0.027700,0.010752,0.007767,0.004840,0.007125,0.015033,0.004892,0.002659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mths_since_last_record:3-20,-0.014320,-0.009835,0.009784,0.009879,0.004840,0.001742,0.002403,0.006023,-0.005617,0.000769,...,-0.004648,-0.025147,0.019007,0.008413,0.003100,1.000000,-0.004793,-0.016567,-0.005396,-0.015849
mths_since_last_record:21-31,-0.020827,-0.004384,0.009233,0.010159,0.007125,0.000164,0.001415,-0.000543,-0.003450,0.003794,...,-0.011759,-0.029075,0.016932,0.019110,0.002409,-0.004793,1.000000,-0.018538,-0.006039,-0.017734
mths_since_last_record:32-80,-0.072079,-0.015251,0.040123,0.030623,0.015033,-0.000684,0.003611,-0.014373,-0.006139,0.014571,...,-0.027440,-0.054037,-0.025230,0.030075,0.099902,-0.016567,-0.018538,1.000000,-0.020872,-0.061299
mths_since_last_record:81-86,-0.023405,-0.003582,0.009529,0.011940,0.004892,0.000945,0.001795,-0.004101,-0.003866,0.006589,...,-0.003966,0.008345,-0.009937,-0.004243,0.006188,-0.005396,-0.006039,-0.020872,1.000000,-0.019967


## **3. Build a Logistic Regression Model with P-Values**

**Zusammenfassung: Variablenselektion im PD-Modell mittels statistischer Signifikanz**

**1. Herausforderung und Zielsetzung**  
Nach der erstmaligen Schätzung des logistischen Regressionsmodells (PD-Modell) bestand die nächste Aufgabe darin, diejenigen unabhängigen Variablen zu identifizieren und zu entfernen, die **keinen statistisch signifikanten Beitrag** zur Vorhersage der Kreditausfallwahrscheinlichkeit leisten. Das Ziel war ein **schlankes und robustes Finalmodell**, das nur die aussagekräftigsten Prädiktoren enthält.

**2. Lösung: Berechnung multivariater p-Werte**  
Da die standardmäßigen p-Werte in `sklearn` univariat sind (d.h. Wechselwirkungen zwischen Variablen ignorieren) und somit für die Bewertung in einem multivariaten Regressionsmodell ungeeignet sind, wurde eine **maßgeschneiderte Lösung** implementiert:  
- Eine neue Klasse `LogisticRegression_with_p_values` wurde erstellt, die von der originalen `LogisticRegression`-Klasse erbt.  
- Diese Klasse überschreibt die `fit`-Methode und berechnet therein **multivariate p-Werte** mittels linearer Algebra.  
- Die p-Werte wurden in einer Instanzvariable `p_values` gespeichert und der Zusammenfassungstabelle der Koeffizienten hinzugefügt.  

**3. Entscheidungskriterium für die Variablenselektion**  
Grundlage für die Entscheidung, eine Variable zu behalten oder zu entfernen, war ihr **p-Wert** (Signifikanzniveau von 0.05). Die Herausforderung bestand darin, dass originale Variablen oft durch **mehrere Dummy-Variablen** repräsentiert werden. Es wurde folgende Regel angewandt:  
- **Alle Dummy-Variablen signifikant:** Behalte die gesamte Variable.  
- **Keine Dummy-Variable signifikant:** Entferne die gesamte Variable.  
- **Einige signifikant, einige nicht:** Aus inhaltlich-konzeptionellen Gründen wurde die **gesamte originale Variable beibehalten**, um eine vollständige Repräsentation aller Ausprägungen im Modell zu gewährleisten. Das Entfernen einzelner Dummies würde zu einer unvollständigen und schwer interpretierbaren Variable führen.  

**4. Durchführung der Selektion und Ergebnis**  
Angewendet auf das initiale Modell führte die Analyse der p-Werte zu folgenden Entscheidungen:  

*Entfernte Variablengruppen (da nicht signifikant):*  
- `delinq_2yrs` (Zahl der Zahlungsrückstände in den letzten 2 Jahren)  
- `open_acc` (Anzahl offener Kreditlinien)  
- `pub_rec` (Anzahl negativer öffentlicher Einträge)  
- `revol_lim` (Kreditlimit für revolving credit)  
- `total_acc` (Gesamtzahl der Kreditkonten)  

*Beibehaltene Variablengruppen (da ganz oder teilweise signifikant):*  
- `grade`, `home_ownership`, `addr_state`, `verification_status`, `purpose`, `initial_list_status`, `term`, `emp_length`, `mths_since_issue_d`, `int_rate`, `mths_since_earliest_cr_line`, `annual_inc`, `dti` sowie andere.  

**5. Das finale Modell**  
- Das **finale Modell** wurde mit dem reduzierten Satz an Variablen neu trainiert.  
- Es umfasst **84 Dummy-Variablen**, die **17 originale unabhängige Variablen** repräsentieren.  
- Dieses Modell ist statistisch robuster, leichter interpretierbar und vermeidet Overfitting, da es nur die Variablen enthält, die einen gesicherten Beitrag zur Vorhersageleistung liefern.  


## Build a Logistic Regression Model with P-Values

In [54]:
# P values for sklearn logistic regression.
# Class to display p-values for logistic regression in sklearn.

from sklearn import linear_model
import scipy.stats as stat

class LogisticRegression_with_p_values:

    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y):
        self.model.fit(X,y)

        #### Get p-values for the fitted model ####
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X / denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores] ### two tailed test for p-values

        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values

In [58]:
bool_cols = inputs_train.select_dtypes(include=['bool']).columns
inputs_train[bool_cols] = inputs_train[bool_cols].astype(int)

In [59]:
reg = LogisticRegression_with_p_values()
# We create an instance of an object from the newly created 'LogisticRegression_with_p_values()' class.

In [60]:
reg.fit(inputs_train, loan_data_targets_train)
# Estimates the coefficients of the object from the 'LogisticRegression' class
# with inputs (independent variables) contained in the first dataframe
# and targets (dependent variables) contained in the second dataframe.

In [61]:
# Same as above.
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficients
0,Intercept,-0.228885
1,grade:A,0.913792
2,grade:B,0.681085
3,grade:C,0.485257
4,grade:D,0.318656
5,grade:E,0.181767
6,grade:F,0.053971
7,home_ownership:OWN,0.083392
8,home_ownership:MORTGAGE,0.118492
9,addr_state:OK_LA_NC_NM_MO_VA_NJ,-0.018838


In [62]:
# This is a list.
p_values = reg.p_values
# We take the result of the newly added method 'p_values' and store it in a variable 'p_values'.

In [63]:
# Add the intercept for completeness.
p_values = np.append(np.nan, np.array(p_values))
# We add the value 'NaN' in the beginning of the variable with p-values.

In [64]:
summary_table['p_values'] = p_values
# In the 'summary_table' dataframe, we add a new column, called 'p_values', containing the values from the 'p_values' variable.

In [65]:
summary_table

,Feature name,Coefficients,p_values
0,Intercept,-0.228885,NaN
1,grade:A,0.913792,9.764635e-26
2,grade:B,0.681085,1.055110e-29
3,grade:C,0.485257,4.681420e-18
4,grade:D,0.318656,2.329082e-09
5,grade:E,0.181767,1.505809e-04
6,grade:F,0.053971,2.804286e-01
7,home_ownership:OWN,0.083392,1.475807e-05
8,home_ownership:MORTGAGE,0.118492,9.141517e-23
9,addr_state:OK_LA_NC_NM_MO_VA_NJ,-0.018838,1.610312e-01


In [66]:
# Vollständige Dezimaldarstellung (kann bei sehr kleinen Werten zu 0.0000 führen)
summary_table['p_values'] = summary_table['p_values'].apply(
    lambda x: f'{x:.6f}' if not pd.isna(x) else 'NaN'
)

# Anzeige der Tabelle
summary_table

,Feature name,Coefficients,p_values
0,Intercept,-0.228885,NaN
1,grade:A,0.913792,0.000000
2,grade:B,0.681085,0.000000
3,grade:C,0.485257,0.000000
4,grade:D,0.318656,0.000000
5,grade:E,0.181767,0.000151
6,grade:F,0.053971,0.280429
7,home_ownership:OWN,0.083392,0.000015
8,home_ownership:MORTGAGE,0.118492,0.000000
9,addr_state:OK_LA_NC_NM_MO_VA_NJ,-0.018838,0.161031


Interpretation der p-Werte
Zur Erinnerung: In der statistischen Analyse:

p-Werte < 0.05 werden generally als statistisch signifikant betrachtet

p-Werte < 0.01 werden als hoch signifikant betrachtet

p-Werte < 0.001 werden als sehr hoch signifikant betrachtet

In Ihrer Tabelle sehen wir, dass viele Variablen extrem signifikante p-Werte haben (z.B., grade:A mit 9.76e-26), was auf eine sehr starke statistische Signifikanz hinweist.

Einige Variablen wie grade:F (0.2804) und delinq_2yrs:>=17 (0.9808) sind nicht signifikant, da ihre p-Werte deutlich über 0.05 liegen.

In [ ]:
####

In [ ]:
import pickle

In [ ]:
pickle.dump(reg2, open('pd_model.sav', 'wb'))
# Here we export our model to a 'SAV' file with file name 'pd_model.sav'.